In [ ]:
## Pretty Output
import warnings
warnings.filterwarnings("ignore")
from rich import pretty
pretty.install()
from rich.traceback import install
install(show_locals=False)
import numpy as np
import time
from tqdm.auto import tqdm
from IPython.display import clear_output
import matplotlib.pyplot as plt
import copy
import gym

In [ ]:
import stable_baselines3
# stable_baselines3.get_system_info()
from stable_baselines3 import DQN, A2C, PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

In [ ]:
from src.main import env_v1, env_v2, GameManager, Game

In [ ]:
games = GameManager(fieldsize=40, fooditems=50)

In [ ]:
id_ = games.newGame()

In [ ]:
check_env_v2 = env_v2(
    "name",
    manager = games,
    game_id = id_,
    others = [],
    viewdistance=10,
    admin=True,
)
check_env(check_env_v2)

In [ ]:
render_env: env_v2 = env_v2(
    "name",
    manager = games,
    game_id = id_,
    others = [],
    viewdistance=10,
    admin=True,
)
env = DummyVecEnv([lambda: render_env])
env: VecNormalize = VecNormalize.load(f"./saved/nrom_for_dqn_best.pkl", env)
#  do not update them at test time
env.training = False
# reward normalization is not needed at test time
# env.norm_reward = False
model = DQN.load(f"./saved/dqn_best", env = env)

In [14]:
from matplotlib import pyplot as plt
# %matplotlib ipympl
from celluloid import Camera
from IPython.display import HTML
# plt.ion()
rewards = []
obs = env.reset()
action = [-1]
fig = plt.figure()
camera = Camera(fig)
for _ in range(1000):
  obs, reward, done, info = env.step(action)
  original_obs = env.get_original_obs()
  # original_obs = original_obs[0, 1]
  print(original_obs)
  print(original_obs.shape)
  action, _states = model.predict(obs, deterministic=True)
  print(action)
  print(action.shape)
  break
  obs = env.get_original_obs()
  reward = env.get_original_reward()
  info = info[0]
  ax = plt.gca()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  game: Game = env.get_attr(attr_name="game")[0]
  O = copy.deepcopy(game.base.field)
  try:
    x = info["x"]
    y = info["y"]
    O[x][y] = 3
  except:
    pass
  plt.imshow(O)
  # plt.show()
  camera.snap()
  rewards.append(reward[0])
  if done:
    obs = env.reset()
    rewards = []
    break
# animation = camera.animate()

[[[[-1 10]
   [ 0 10]
   [ 0 10]
   [ 1  7]
   [ 1  2]
   [-1 10]
   [ 0 10]
   [-1 10]
   [ 1  1]]

  [[ 0  0]
   [ 0  0]
   [ 0  0]
   [ 0  0]
   [ 0  0]
   [ 0  0]
   [ 0  0]
   [ 0  0]
   [ 0  0]]]]
(1, 2, 9, 2)
[3]
(1,)


In [ ]:
HTML(animation.to_html5_video())